In [15]:
#Import CNN Libaries (tensorflow.keras)

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization, Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [16]:
# Import Decision Tree libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [17]:
class SegmentationClassificationModel: 
    
    def __init__(self, X_train, y_train, X_test, y_test, X_val, y_val):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.X_val = X_val
        self.y_val = y_val
        